<h5>Data cleaning - Part 2</h5>
<h6></h6>

In [40]:
!pip install ipython-sql
!pip install sqlalchemy
!pip install us
%load_ext sql
db_path = "Food_Recall.db"
%sql sqlite:///{db_path}

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [41]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re #import regular expressions module (set of string that matches the criteria)
from collections import defaultdict
import us #import us library
fda = pd.read_csv("Food_Recall.csv")
conn = sqlite3.connect("Food_Recall.db")
fda.to_sql("Food_Recall", conn, if_exists="replace", index=False)

27526

In [42]:
#inspecting data using Python
#display data first 5 rows
fda.head()

,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Product Description,Event ID,Event Classification,Product ID,Center,Recall Details
0,3004312676,HAR Maspeth Corp.,Food/Cosmetics,Class I,Ongoing,"NY, NJ, MA, CT, PA, MD, NC, IL, MI, GA",Maspeth,New York,United States,3/21/2025,Contains undeclared egg,Jinga Glass Noodles with vegetables 8 oz. and ...,96503,Class I,213039,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...
1,2938745,Lyons Magnus LLC,Food/Cosmetics,Class I,Ongoing,Distribution centers located throughout the U....,Fresno,California,United States,3/20/2025,Potential contamination with Listeria monocyto...,"Ready Care Chocolate Shake (4 oz), UPC 1004579...",96376,Class I,212741,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...
2,2938745,Lyons Magnus LLC,Food/Cosmetics,Class I,Ongoing,Distribution centers located throughout the U....,Fresno,California,United States,3/20/2025,Potential contamination with Listeria monocyto...,"Ready Care Strawberry Shake (4 oz), UPC 100457...",96376,Class I,212977,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...
3,2938745,Lyons Magnus LLC,Food/Cosmetics,Class I,Ongoing,Distribution centers located throughout the U....,Fresno,California,United States,3/20/2025,Potential contamination with Listeria monocyto...,"Ready Care Vanilla Shake (4 oz), UPC 100457960...",96376,Class I,212978,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...
4,2938745,Lyons Magnus LLC,Food/Cosmetics,Class I,Ongoing,Distribution centers located throughout the U....,Fresno,California,United States,3/20/2025,Potential contamination with Listeria monocyto...,Ready Care Chocolate Shake No Sugar Added (4 o...,96376,Class I,212982,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...


In [43]:
#checking for missing values
fda.isnull().sum()

FEI Number                    0
Recalling Firm Name           0
Product Type                  0
Product Classification        0
Status                        0
Distribution Pattern          0
Recalling Firm City           0
Recalling Firm State          0
Recalling Firm Country        0
Center Classification Date    0
Reason for Recall             0
Product Description           0
Event ID                      0
Event Classification          0
Product ID                    0
Center                        0
Recall Details                0
dtype: int64

In [44]:
#check for data types and info()
fda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27526 entries, 0 to 27525
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   FEI Number                  27526 non-null  object
 1   Recalling Firm Name         27526 non-null  object
 2   Product Type                27526 non-null  object
 3   Product Classification      27526 non-null  object
 4   Status                      27526 non-null  object
 5   Distribution Pattern        27526 non-null  object
 6   Recalling Firm City         27526 non-null  object
 7   Recalling Firm State        27526 non-null  object
 8   Recalling Firm Country      27526 non-null  object
 9   Center Classification Date  27526 non-null  object
 10  Reason for Recall           27526 non-null  object
 11  Product Description         27526 non-null  object
 12  Event ID                    27526 non-null  int64 
 13  Event Classification        27526 non-null  ob

In [45]:
#checking each columns for anomalies by looking at the number of unique values
fda.nunique()

FEI Number                     4599
Recalling Firm Name            4914
Product Type                      1
Product Classification            3
Status                            3
Distribution Pattern           5777
Recalling Firm City            1969
Recalling Firm State             53
Recalling Firm Country           33
Center Classification Date     2654
Reason for Recall              7809
Product Description           27379
Event ID                       7230
Event Classification              3
Product ID                    27526
Center                            1
Recall Details                27526
dtype: int64

In [46]:
#drop unrelated columns
fda_1 = fda.drop(columns = ["Center", "Recall Details"])

#Remove duplicate rows
fda_1 = fda_1.drop_duplicates(subset="FEI Number")

#Filtering out "cosmetic" products
fda_1 = fda_1[~fda["Product Description"].str.contains("cosmetic" or "shampoo", case = False, na = False)]
fda_1 = fda_1[~fda["Reason for Recall"].str.contains("cosmetic" or "shampoo", case = False, na = False)]

fda_1.head()

C:\Users\jukji\AppData\Local\Temp\ipykernel_38788\1028511969.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fda_1 = fda_1[~fda["Product Description"].str.contains("cosmetic" or "shampoo", case = False, na = False)]
C:\Users\jukji\AppData\Local\Temp\ipykernel_38788\1028511969.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fda_1 = fda_1[~fda["Reason for Recall"].str.contains("cosmetic" or "shampoo", case = False, na = False)]


,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Product Description,Event ID,Event Classification,Product ID
0,3004312676,HAR Maspeth Corp.,Food/Cosmetics,Class I,Ongoing,"NY, NJ, MA, CT, PA, MD, NC, IL, MI, GA",Maspeth,New York,United States,3/21/2025,Contains undeclared egg,Jinga Glass Noodles with vegetables 8 oz. and ...,96503,Class I,213039
1,2938745,Lyons Magnus LLC,Food/Cosmetics,Class I,Ongoing,Distribution centers located throughout the U....,Fresno,California,United States,3/20/2025,Potential contamination with Listeria monocyto...,"Ready Care Chocolate Shake (4 oz), UPC 1004579...",96376,Class I,212741
11,3030625663,Prep'd CDA LLC,Food/Cosmetics,Class II,Ongoing,Distributed in ID and WA,Coeur D Alene,Idaho,United States,3/20/2025,Undeclared soy lecithin and undeclared color a...,Chocolate Peanut Butter Protein Balls. Product...,96385,Class II,212743
12,3008647169,HENKEL CAPITAL SA DE CV,Food/Cosmetics,Class II,Ongoing,Henkel distributed the recalled products to 3 ...,Huisquilucan,-,Mexico,3/20/2025,(Imported) Shampoo product tested positive for...,Tec Italy branded shampoo conditioner for dry ...,96436,Class II,212870
13,3014064807,Growland Inc,Food/Cosmetics,Class II,Ongoing,"Product was distributed to customers in CA, TX...",Chino,California,United States,3/20/2025,Elevated levels of Sulfites in Dried Longan Pulp,"Dried Longan Pulp- Made in China, 8oz(227g), G...",96264,Class II,212459


In [52]:
#create a new column "Reason Category" to categorize common reasons for recall
#define patterns (key/value dictionary)
patterns = {
    "Undeclared": r"(undeclared|declare|declaration)",
    "Listeria": r"(listeria|L. mono|monocytogenes)",
    "Salmonella": r"(salmonella|salmonellosis)",
    "Pseudomonas": r"(pseudomonas)",
    "Mold": r"(mold)",
    "E-Coli": r"(e-coli|e coli|e. coli)" #match e-coli or e coli
}

#create a new column and default to 'Others'
fda_1["Reason Category"] = "Others"

#categorize based on the pattern matching
for category, pattern in patterns.items():
    fda_1.loc[fda_1["Reason for Recall"].str.contains(pattern, case=False, na=False), "Reason Category"] = category

fda_1["Reason Category"].value_counts()

C:\Users\jukji\AppData\Local\Temp\ipykernel_38788\4201146946.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  fda_1.loc[fda_1["Reason for Recall"].str.contains(pattern, case=False, na=False), "Reason Category"] = category


,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Nationwide,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Reason Category,Product Description,Event ID,Event Classification,Product ID
0,3004312676,HAR Maspeth Corp.,Food/Cosmetics,Class I,Ongoing,"NY, NJ, MA, CT, PA, MD, NC, IL, MI, GA",No,Maspeth,New York,United States,2025-03-21,Contains undeclared egg,Undeclared,Jinga Glass Noodles with vegetables 8 oz. and ...,96503,Class I,213039
1,2938745,Lyons Magnus LLC,Food/Cosmetics,Class I,Ongoing,Distribution centers located throughout the U....,Yes,Fresno,California,United States,2025-03-20,Potential contamination with Listeria monocyto...,Listeria,"Ready Care Chocolate Shake (4 oz), UPC 1004579...",96376,Class I,212741
11,3030625663,Prep'd CDA LLC,Food/Cosmetics,Class II,Ongoing,Distributed in ID and WA,No,Coeur D Alene,Idaho,United States,2025-03-20,Undeclared soy lecithin and undeclared color a...,Undeclared,Chocolate Peanut Butter Protein Balls. Product...,96385,Class II,212743
12,3008647169,HENKEL CAPITAL SA DE CV,Food/Cosmetics,Class II,Ongoing,Henkel distributed the recalled products to 3 ...,No,Huisquilucan,-,Mexico,2025-03-20,(Imported) Shampoo product tested positive for...,Others,Tec Italy branded shampoo conditioner for dry ...,96436,Class II,212870
13,3014064807,Growland Inc,Food/Cosmetics,Class II,Ongoing,"Product was distributed to customers in CA, TX...",No,Chino,California,United States,2025-03-20,Elevated levels of Sulfites in Dried Longan Pulp,Others,"Dried Longan Pulp- Made in China, 8oz(227g), G...",96264,Class II,212459


In [54]:
#create a nationwide column
fda_1["Nationwide"] = np.nan
nationwide = r"(nationwide|throughout the U.S.|thru out the United States.)"
fda_1.loc[fda_1["Distribution Pattern"].str.contains(nationwide, case=False, na=False), "Nationwide"] = "Yes"
fda_1.loc[fda_1["Nationwide"].isnull(), "Nationwide"] = "No"

C:\Users\jukji\AppData\Local\Temp\ipykernel_38788\3668327310.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  fda_1.loc[fda_1["Distribution Pattern"].str.contains(nationwide, case=False, na=False), "Nationwide"] = "Yes"
C:\Users\jukji\AppData\Local\Temp\ipykernel_38788\3668327310.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Yes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  fda_1.loc[fda_1["Distribution Pattern"].str.contains(nationwide, case=False, na=False), "Nationwide"] = "Yes"


In [55]:
#Update date format
fda_1["Center Classification Date"] = pd.to_datetime(fda_1["Center Classification Date"])
fda_1["Year"] = fda_1["Center Classification Date"].dt.year
fda_1["Month"] = fda_1["Center Classification Date"].dt.month

In [56]:
#reorganize the columns
new_column_order = [
    "FEI Number", "Recalling Firm Name", "Product Type", "Product Classification",
    "Status", "Distribution Pattern", "Nationwide", "Recalling Firm City", 
    "Recalling Firm State", "Recalling Firm Country", "Center Classification Date", 
    "Reason for Recall", "Reason Category", "Product Description", "Event ID", 
    "Event Classification", "Product ID"
]

fda_1 = fda_1[new_column_order]

In [59]:
#reset index
fda_1.reset_index(drop=True, inplace=True)

,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Nationwide,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Reason Category,Product Description,Event ID,Event Classification,Product ID
0,3004312676,HAR Maspeth Corp.,Food/Cosmetics,Class I,Ongoing,"NY, NJ, MA, CT, PA, MD, NC, IL, MI, GA",No,Maspeth,New York,United States,2025-03-21,Contains undeclared egg,Undeclared,Jinga Glass Noodles with vegetables 8 oz. and ...,96503,Class I,213039
1,2938745,Lyons Magnus LLC,Food/Cosmetics,Class I,Ongoing,Distribution centers located throughout the U....,Yes,Fresno,California,United States,2025-03-20,Potential contamination with Listeria monocyto...,Listeria,"Ready Care Chocolate Shake (4 oz), UPC 1004579...",96376,Class I,212741
2,3030625663,Prep'd CDA LLC,Food/Cosmetics,Class II,Ongoing,Distributed in ID and WA,No,Coeur D Alene,Idaho,United States,2025-03-20,Undeclared soy lecithin and undeclared color a...,Undeclared,Chocolate Peanut Butter Protein Balls. Product...,96385,Class II,212743
3,3008647169,HENKEL CAPITAL SA DE CV,Food/Cosmetics,Class II,Ongoing,Henkel distributed the recalled products to 3 ...,No,Huisquilucan,-,Mexico,2025-03-20,(Imported) Shampoo product tested positive for...,Others,Tec Italy branded shampoo conditioner for dry ...,96436,Class II,212870
4,3014064807,Growland Inc,Food/Cosmetics,Class II,Ongoing,"Product was distributed to customers in CA, TX...",No,Chino,California,United States,2025-03-20,Elevated levels of Sulfites in Dried Longan Pulp,Others,"Dried Longan Pulp- Made in China, 8oz(227g), G...",96264,Class II,212459


In [6]:
#Count each state once per row even if both abbreviation and full name appear
#Count either abbreviation or full name, not both, only one per state per row
#Sum that up across all rows for each state

#create dictionary of state abbreviations and full names
states = {state.abbr: state.name for state in us.states.STATES}

#Store in count number
state_counts = defaultdict(int)

#loop through each state abbreviation and full name
for pattern in df['Distribution Pattern'].dropna():
    for abbr, full in states.items():
        if (re.search(r'\b' + abbr + r'\b', pattern, flags = re.IGNORECASE) or
             re.search(r'\b' + full + r'\b', pattern, flags = re.IGNORECASE)):
              state_counts[full] += 1

#Get a count of how many rows mention each state
#Print output line by line, more clean
for full, count in sorted(state_counts.items()):
    print(f"{full}: {count}")

Alabama: 3637
Alaska: 772
Arizona: 4548
Arkansas: 3178
California: 7455
Colorado: 4133
Connecticut: 4267
Delaware: 2259
Florida: 7010
Georgia: 5560
Hawaii: 1282
Idaho: 2079
Illinois: 7233
Indiana: 11629
Iowa: 3666
Kansas: 3181
Kentucky: 4511
Louisiana: 3458
Maine: 2558
Maryland: 4990
Massachusetts: 4735
Michigan: 6162
Minnesota: 4528
Mississippi: 2951
Missouri: 5120
Montana: 2070
Nebraska: 2626
Nevada: 3378
New Hampshire: 2783
New Jersey: 5874
New Mexico: 2106
New York: 8009
North Carolina: 6053
North Dakota: 1700
Ohio: 7476
Oklahoma: 3184
Oregon: 4862
Pennsylvania: 7569
Rhode Island: 1883
South Carolina: 4343
South Dakota: 1434
Tennessee: 4631
Texas: 6711
Utah: 2464
Vermont: 1138
Virginia: 6401
Washington: 4918
West Virginia: 1959
Wisconsin: 5570
Wyoming: 2083
